# Converting Scraped Data into IDPCentral

Alasdair J G Gray ([ORCID:0000-0002-5711-4872](http://orcid.org/0000-0002-5711-4872))

_Heriot-Watt University, Edinburgh, UK_

## Introduction

IDPCentral is the idea of having a central registry of proteins that are known to be disordered.

We aim to populate the content of the registry with Bioschemas markup that has been scraped using the BMUSE tool.

This notebook goes through the steps of converting the scraped content into the IDPCentral data model.

## IDPCentral Data Model

The IDPCental data model reuses ideas from [Wikidata](https://www.mediawiki.org/wiki/Wikibase/DataModel) whereby every statement loaded contains a provenance link as to where it was acquired.

- [ ] Document IDPCentral Model

## Data Sources

The following databases have been scraped to populate IDPCentral
- [DisProt](https://www.disprot.org/)
- [MobiDb](https://mobidb.bio.unipd.it/)
- [Protein Ensemble Database](https://proteinensemble.org/)

## Python Libraries

We need some support libraries to allow us to interface with a SPARQL endpoint and to process the results. We also need `Graph` from `rdflib` to process the responses from CONSTRUCT queries.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from rdflib import Graph

In [ ]:
from string import Template

## DisProt Conversion

First we configure the SPARQL endpoint where we have loaded the data and setup a function that will query the endpoint and return the results in JSON.

In [ ]:
dpSparql = SPARQLWrapper("http://localhost:3030/disprot/sparql")
def dpSelect(query):
    dpSparql.setReturnFormat(JSON)
    dpSparql.setQuery(query)
    results = dpSparql.queryAndConvert()
    return results
def dpConstruct(query):
    dpSparql.setReturnFormat(N3)
    dpSparql.setOnlyConneg(True)
    dpSparql.setQuery(query)
    results = dpSparql.query().convert()
    g = Graph()
    g.parse(data=results, format="n3")
    return g

In [ ]:
convertQuery = Template("""
PREFIX schema: <https://schema.org/>
CONSTRUCT {
    <${uniprotIRI}> a schema:Protein ;
        schema:identifier ?identifier ;
        schema:name ?name ;
        schema:hasSequenceAnnotation ?annotation ;
        schema:taxonomicRange ?taxonomicRange ;
        schema:hasRepresentation ?representation ;
        schema:sameAs ?sameAs , <${disprotIRI}>.
}
WHERE {
    GRAPH ?g {
# Bioschemas Minimal Properties
        <${disprotIRI}> a schema:Protein ;
            schema:identifier ?identifier ;
            schema:name ?name ;
# Bioschemas Recommended properties
            schema:hasSequenceAnnotation ?annotation ;
            schema:taxonomicRange ?taxonomicRange ;
# Bioschemas Optional properties
            schema:hasRepresentation ?representation ;
            schema:sameAs ?sameAs .
    }
}
""")
def convertDisprot(disprot, uniprot):
    query = convertQuery.substitute(disprotIRI=disprot,uniprotIRI=uniprot)
    #print(query)
    results = dpConstruct(query)
    return results


In [ ]:
query = """
PREFIX schema: <https://schema.org/>
SELECT ?disprot ?uniprot
WHERE {
    GRAPH ?g {
        ?disprot a schema:Protein ;
            schema:sameAs ?uniprot .
        FILTER regex(str(?uniprot), "^https://www.uniprot.org/uniprot/")
    }
}
"""
results = dpSelect(query)
for result in results["results"]["bindings"]:
    disprot = result["disprot"]["value"]
    uniprot = result["uniprot"]["value"]
    response = convertDisprot(disprot, uniprot)
    print(response.serialize(format='nt'))

## Conversion using RDFlib

This is an attempt to achieve the same functionality without using a triplestore.

Load in the RDFLib library.

In [85]:
from rdflib import ConjunctiveGraph
from rdflib.plugins.sparql.processor import prepareQuery

In [86]:
from string import Template

Read in data file.

In [91]:
g = ConjunctiveGraph()
#g.parse("2242.nq", format="nquads")
g.parse("../scraped-data/disprot/2242.nq", format="nquads")

<Graph identifier=file:///Users/ajg33/Workspace/BioHackathon-projects-2020/projects/24/IDPCentral/scraped-data/disprot/2242.nq (<class 'rdflib.graph.Graph'>)>

Prepare query to extract UniProt and DisProt IRIs.

In [46]:
idQuery = prepareQuery(
"""
PREFIX schema: <https://schema.org/>
SELECT ?disprot ?uniprot
WHERE {
    GRAPH ?g {
        ?disprot a schema:Protein ;
            schema:sameAs ?uniprot .
        FILTER regex(str(?uniprot), "^https://www.uniprot.org/uniprot/")
    }
}
"""
)

Prepare query for doing the conversion.

In [48]:
convertQuery = prepareQuery("""
PREFIX schema: <https://schema.org/>
CONSTRUCT {
    ?uniprotIRI a schema:Protein ;
        schema:identifier ?identifier ;
        schema:name ?name ;
        schema:hasSequenceAnnotation ?annotation ;
        schema:taxonomicRange ?taxonomicRange ;
        schema:hasRepresentation ?representation ;
        schema:sameAs ?sameAs , ?disprotIRI.
}
WHERE {
    GRAPH ?g {
# Bioschemas Minimal Properties
        ?disprotIRI a schema:Protein ;
            schema:identifier ?identifier ;
            schema:name ?name ;
# Bioschemas Recommended properties
            schema:hasSequenceAnnotation ?annotation ;
            schema:taxonomicRange ?taxonomicRange ;
# Bioschemas Optional properties
            schema:hasRepresentation ?representation ;
            schema:sameAs ?sameAs .
    }
}
""")

In [82]:
def convertDisprot(disprot, uniprot):
    print('DisProt: ' + disprot + ' ' + type(disprot))
    print("hello")
    
#     result_graph = g.query(convertQuery, initBindings={'disprotIRI': disprot,'uniprotIRI': uniprot})
#     for i in ['askAnswer', 'bindings', 'graph',
#            'selectionF', 'type', 'vars']:
#          v = getattr(results, i)
#          print(i, type(v), v, repr(v))

#     return results

Extract UniProt and DisProt IRIs, then use them to convert the data to the IDPCentral model

In [98]:
results = g.query(idQuery)

# for result in results:
#     print(result)
#     print("Disprot: %s\nUniProt: %s" % row)    

for result in results:
    response = convertDisprot(result['disprot'], result['uniprot'])
# #     print(response.serialize(format='nt'))



DisProt: http://disprot.org/DP00003 does not look like a valid URI, trying to serialize this will break.
DisProt: http://disprot.org/DP00003  does not look like a valid URI, trying to serialize this will break.


(rdflib.term.URIRef('http://disprot.org/DP00003'), rdflib.term.URIRef('https://www.uniprot.org/uniprot/P03265'))
Disprot: http://disprot.org/DP00003
UniProt: https://www.uniprot.org/uniprot/P03265
(rdflib.term.URIRef('http://disprot.org/DP00003'), rdflib.term.URIRef('https://www.uniprot.org/uniprot/P03265'))
Disprot: http://disprot.org/DP00003
UniProt: https://www.uniprot.org/uniprot/P03265


TypeError: must be str, not type